In [18]:
'''Predicting Mortality across Germany wih different AI Methods
'''

import numpy as np
import pandas as pd
import os
from matplotlib import pyplot as plt
import sklearn
from sklearn.model_selection import train_test_split
from sklearn import impute
from sklearn import linear_model
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from sklearn.ensemble import RandomForestRegressor, AdaBoostRegressor, GradientBoostingRegressor, VotingRegressor
import dbf
from pygam import GAM, s, f, LinearGAM
import xgboost
import warnings
warnings.filterwarnings('ignore')
%matplotlib inline

### Load Data

In [19]:
# Set the working directory
os.chdir("N:/WG_ENRI/20_projects/Noise2NAKO/04_data/grid_prediction/3_output")

In [30]:
# Read training and test data
data_training = pd.read_csv('Prediction/Training_data_cvd_noise.csv', sep=',')
data_test = pd.read_csv('Prediction/Test_data_cvd_noise.csv', sep=',')

In [35]:
# distinguish between response and input variables and make training and test data 
# remove unnecessary features 'id', 'x_sw', 'y_sw','x_mp', 'y_mp'

output_variable = 'cvd_mortality_17'
output_path = 'CVD Mortality Rate 2017 incl Noise_XY'

# output_variable = 'ks_Mortality_17'
# output_path = 'Total Mortality Rate 2017 incl XY'

output_path = '../5_results/' + output_path + '/'
coord = True

y = data_training[output_variable]
if coord:
    X = data_training.drop([output_variable, 'ks_Mortality_17', 'id'], axis = 1)   
else:
    X = data_training.drop([output_variable, 'ks_Mortality_17', 'id', 'x_sw', 'y_sw','x_mp', 'y_mp'], axis = 1)
    
pred_y = data_test[output_variable]
if coord:
    pred_X = data_test.drop([output_variable, 'ks_Mortality_17', 'id'], axis = 1)
else:
    pred_X = data_test.drop([output_variable, 'ks_Mortality_17', 'id', 'x_sw', 'y_sw','x_mp', 'y_mp'], axis = 1)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 10)

# Prediction

In [47]:
def Prediction(method, X_train, X_test, y_train, y_test, pred_X, pred_y, output_variable, output_path, data_training, data_test):
    
    '''Prediction Function'''
    
    if method == 'LR':
        model = linear_model.LinearRegression().fit(X_train, y_train)
    elif method == 'LR_Ridge':
        model = linear_model.RidgeCV(cv = 5).fit(X_train, y_train)
    elif method == 'LR_Lasso':
        model = linear_model.LassoCV(cv = 5, random_state=0).fit(X_train, y_train)
    elif method == 'LR_Elastic':
        model = linear_model.ElasticNetCV(cv = 5, random_state=0).fit(X_train, y_train)
    elif method == 'GAM':
        model = GAM(s(0) + s(1) + s(2) + s(3) + s(4) + s(5) + s(6) + s(7) + s(8) + s(9)).fit(X_train, y_train)
    elif method == 'RF':
        model = RandomForestRegressor(n_estimators= 10, max_depth= 10, random_state= 0, max_features='sqrt').fit(X_train, y_train)
    elif method == 'AdaB':
        model = AdaBoostRegressor(n_estimators= 10, random_state=0).fit(X_train, y_train)
    else:
        model = xgboost.XGBRegressor().fit(X_train, y_train)
    
    y_pred_train = model.predict(X_train)
    y_pred_test = model.predict(X_test)
    y_pred = model.predict(pred_X)
    y_pred = pd.DataFrame(y_pred, columns = [output_variable])

    result = {'Method': method, 'MSE_train': mean_squared_error(y_train, y_pred_train),
                            'MAE_train': mean_absolute_error(y_train, y_pred_train), 'R_2_train': r2_score(y_train, y_pred_train),
                           'MSE_val': mean_squared_error(y_test, y_pred_test), 'MAE_val': mean_absolute_error(y_test, y_pred_test),
                           'R_2_val': r2_score(y_test, y_pred_test), 'MSE_pred': mean_squared_error(pred_y, y_pred), 'MAE_pred': mean_absolute_error(pred_y, y_pred),
                           'R_2_pred': r2_score(pred_y, y_pred)}
    
#     Merge dataframes to make the final output dataset 
    data_result= pd.DataFrame()
    data_result = data_result.append(data_training)
    data_result = data_result.append(pd.concat([data_test.drop([output_variable], axis = 1),pd.DataFrame(y_pred)],axis=1))
    data_result = data_result[['id', output_variable]]
    data_result.to_csv((output_path + 'prediction_result_'+method+'.csv'), sep=',', index=False)
    
#     Merge dataframes to make the Ground Truth
    data_result= pd.DataFrame()
    data_result = data_result.append(data_training)
    data_result = data_result.append(pd.concat([data_test.drop([output_variable], axis = 1),pd.DataFrame(pred_y)],axis=1))
    data_result = data_result[['id', output_variable]]
    data_result.to_csv((output_path + 'prediction_result_GT.csv'), sep=',', index=False)
    
    
#     Make the difference output
    data_result= pd.DataFrame()
    data_result = data_result.append(data_training)
    data_result[output_variable] = 0
    pred_variable = 'pred'+output_variable
    data_test = pd.concat([data_test, y_pred.rename(columns={output_variable: pred_variable})], axis=1)
    data_test[output_variable] = data_test[output_variable] - data_test[pred_variable]
    data_result = data_result.append(data_test.drop([pred_variable], axis = 1))
    data_result = data_result[['id', output_variable]]
    data_result.to_csv((output_path + 'prediction_result_diff_'+method+'.csv'), sep=',', index=False)
    
    return result, y_pred

### Prediction with different methods

In [48]:
result= pd.DataFrame(columns=['Method', 'MSE_train', 'MAE_train', 'R_2_train', 'MSE_val', 'MAE_val', 'R_2_val', 'MSE_pred', 'MAE_pred', 'R_2_pred'])

methods = ['LR', 'LR_Ridge', 'LR_Lasso', 'LR_Elastic', 'GAM', 'RF', 'AdaB', 'XGBoost']

for i in methods:
    ToAppend, y_pred_LR = Prediction(i, X_train, X_test, y_train, y_test, pred_X, pred_y, output_variable,
                                 output_path, data_training, data_test)
    result = result.append(ToAppend, ignore_index=True)

result.to_csv(output_path + 'PredictionTask_results_alMethods.csv', index=False, sep=',')
result.round(decimals=3).to_csv(output_path + 'PredictionTask_results_alMethods_round.csv', index=False, sep=',')

result

,Method,MSE_train,MAE_train,R_2_train,MSE_val,MAE_val,R_2_val,MSE_pred,MAE_pred,R_2_pred
0,LR,0.109875,0.239581,0.895079,0.098786,0.223477,0.903112,0.506337,0.540181,0.499332
1,LR_Ridge,0.110552,0.240676,0.894433,0.100105,0.227555,0.901819,0.520690,0.551779,0.485140
2,LR_Lasso,0.518194,0.589722,0.505173,0.463448,0.570030,0.545459,0.862006,0.685550,0.147645
3,LR_Elastic,0.518198,0.589725,0.505169,0.463451,0.570032,0.545457,0.862007,0.685550,0.147644
4,GAM,0.019731,0.088177,0.981159,0.029679,0.106747,0.970892,0.470500,0.507509,0.534768
5,RF,0.009992,0.050354,0.990458,0.032427,0.097554,0.968197,0.436365,0.502720,0.568521
6,AdaB,0.048115,0.157629,0.954055,0.063676,0.175032,0.937548,0.478524,0.531680,0.526834
7,XGBoost,0.000002,0.000940,0.999998,0.013894,0.055238,0.986373,0.397873,0.474731,0.606581
